In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

data_dir = "/home/solomon/Downloads/mnist"
mnist = input_data.read_data_sets(data_dir, one_hot=True)

Extracting /home/solomon/Downloads/mnist/train-images-idx3-ubyte.gz
Extracting /home/solomon/Downloads/mnist/train-labels-idx1-ubyte.gz
Extracting /home/solomon/Downloads/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/solomon/Downloads/mnist/t10k-labels-idx1-ubyte.gz


In [15]:
from tensorflow.python.framework.ops import Tensor
from tensorflow.python.training.moving_averages import ExponentialMovingAverage

In [27]:
def inference_with_scope(input_tensor:Tensor, output_tensor_nodes:int,
                         wname:str, bname:str, scope:str="layer", reuse:bool=False,
                         avg_class:ExponentialMovingAverage=None):
    input_shape = input_tensor.shape.as_list()
    with tf.variable_scope(scope, reuse=reuse):
        weight = tf.get_variable(wname, [input_shape[-1], output_tensor_nodes],
                                 initializer=tf.truncated_normal_initializer(stddev=0.1))
        biases = tf.get_variable(bname, [output_tensor_nodes],
                                 initializer=tf.constant_initializer(0.1))
    if avg_class is None:
        return tf.matmul(input_tensor, weight) + biases
    else:
        return tf.matmul(input_tensor, avg_class.average(weight)) + avg_class.average(biases)

In [28]:
def inference(input_tensor:Tensor, output_tensor_nodes:int,
              avg_class:ExponentialMovingAverage=None):
    input_shape = input_tensor.shape.as_list()
    
    weight = tf.Variable(tf.truncated_normal([input_shape[-1], output_tensor_nodes], stddev=0.1))
    biases = tf.Variable(tf.constant(0.1, shape=[output_tensor_nodes]))
    
    if avg_class is None:
        return tf.matmul(input_tensor, weight) + biases
    else:
        return tf.matmul(input_tensor, avg_class.average(weight)) + avg_class.average(biases)

In [60]:
def f_mnist_0(mnist):
    with tf.Session() as sess:
        x = tf.placeholder(tf.float32, [None, 784], name="x-input")
        layer1 = tf.nn.relu(inference(x, 50))
        y = inference(layer1, 10)
        
        y_ = tf.placeholder(tf.float32, [None, 10], name="y-input")
        
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.argmax(y_, 1), logits=y)
        cross_entropy_mean = tf.reduce_mean(cross_entropy)
        
        train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy_mean)
        
        #print([i.name for i in tf.trainable_variables()])
        print(y.name)
        
        tf.global_variables_initializer().run()

        # Train
        for _ in range(1000):
            batch_xs, batch_ys = mnist.train.next_batch(100)
            sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

        # Test trained model
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                            y_: mnist.test.labels}))

f_mnist_0(mnist)

add_67:0
0.9618


In [57]:
?tf.get_variable